In [ ]:
import os
import subprocess
import re
from ultralytics import YOLO

In [ ]:
def get_model_size_MB(model_path):
    return os.path.getsize(model_path) / (1024 * 1024) 

In [ ]:
def get_parameters_base_model(model):
    total_capas = sum(1 for _ in model.modules())
    total_parametros = sum(p.numel() for p in model.parameters())
    #summary(model, (3,224,224)) ## summary modelo pth o pt segun pytorch
    return total_capas, total_parametros

## dependiendo del nombre del env, en mi caso "salmons" recuerda instalar pip install polygraphy para poder contar las capas y parametros del engine

In [ ]:
#get layer of an .eninge or a .onnx model
def get_layers(is_trt, model_path):
    # para que funcione como sudo es necesario correr desde el path del enviroment env/bin/polygraphy
    if is_trt:
        cmd = f"../../salmons/bin/polygraphy inspect model {model_path}"
    else:
        cmd = f"../../salmons/bin/polygraphy inspect model {model_path} --display-as=trt"

    # Ejecuta el comando y captura la salida
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Decodifica la salida a texto
    output = stdout.decode()

    # Usa una expresión regular para encontrar el número de capas
    match = re.search(r"---- (\d+) Layer\(s\) ----", output)
    # Extrae el número de capas si se encuentra el patrón
    if match:
        num_layers = int(match.group(1))
        return num_layers
    else:
        print("No se encontró el número de capas")
        return 0

In [ ]:
# get parameters from engine or onnx model
def get_parametros(is_trt, model_path):
    if is_trt:
        cmd = f"../../salmons/bin/python ../../utils/param_counter.py --engine {model_path}"
    else:
        cmd = f"../../salmons/bin/onnx_opcounter {model_path}"

    # Ejecuta el comando y captura la salida
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()

    # Decodifica la salida a texto
    output = stdout.decode()

    # Usa una expresión regular para encontrar el número de capas
    match = re.search(r"Number of parameters in the model: (\d+)", output)
    if match:
        num_parameters = int(match.group(1))
        return num_parameters
    else:
        print("No se encontró el número de parametros")
        return 0

In [ ]:
model_path = '../../weights/yolov8lsalmons.pt'
print("Model Size = ", get_model_size_MB(model_path))
print("# Layers = ", get_parameters_base_model(YOLO(model_path))[0], "\n# Parameters = ", get_parameters_base_model(YOLO(model_path))[1])

In [ ]:
model_path = '../weights/yolov8lsalmons_fp32.engine'
print("TRT fp32")
print("Model Size = ", get_model_size_MB('../'+model_path))
print("# Layers = ", get_layers(True,'../'+model_path), "\n# Parameters = ", get_parametros(True, model_path))

In [ ]:
model_path = '../weights/yolov8lsalmons_fp16_bs32_oplvl3.engine'
print("TRT fp16")
print("Model Size = ", get_model_size_MB('../'+model_path))
print("# Layers = ", get_layers(True,'../'+model_path), "\n# Parameters = ", get_parametros(True, model_path))

In [ ]:
model_path = '../weights/yolov8lsalmons_int8_bs32.engine'
print("TRT int8")
print("Model Size = ", get_model_size_MB('../'+model_path))
print("# Layers = ", get_layers(True,'../'+model_path), "\n# Parameters = ", get_parametros(True, model_path))